# 搜索算法
深度优先搜索和广度优先搜索是两种最常见的优先搜索方法，它们被广泛地运用在图和树等结构中进行搜索
- 深度优先(DFS)：深度优先搜索也可以用来检测环路,有时我们可能会需要对已经搜索过的节点进行标记，以防止在遍历时重复搜索某个节点，这种做法叫做‘状态记录’或‘记忆化’。先入后出的栈来实现，也可以通过与栈等价的递归来实现。
- 广度优先(BFS): 采用队列进行

## 问题1：[岛屿的最大面积](https://leetcode-cn.com/problems/max-area-of-island/)
给定一个二维的 0-1 矩阵，其中 0 表示海洋，1 表示陆地。单独的或相邻的陆地可以形成岛屿，每个格子只与其上下左右四个格子相邻。求最大的岛屿面积.

分析：
- 深度优先搜索类型的题可以分为主函数和辅函数，主函数用于遍历所有的搜索位置，判断是否可以开始搜索，如果可以即在辅函数进行搜索。辅函数则负责深度优先搜索的递归调用(这里需要判定上下左右因此有四个分支的调用)。
- 这里使用沉岛思想，遍历过的1沉下去，这样之后就不会再被遍历到了。

In [ ]:
def dfs(x, y, arr):
    if x<0 or y<0 or x >= len(arr) or y >= len(arr[0]) or arr[x][y] == 0:
        return 0
    arr[x][y] = 0   # 把1沉掉
    num = 1         # 来到这一步说明已经登陆了
    num += dfs(x-1, y, arr)
    num += dfs(x+1, y, arr)
    num += dfs(x, y+1, arr)
    num += dfs(x, y-1, arr)
    return num

def process(arr):
    ans = 0
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            ans = max(ans, dfs(i, j, arr)) # 每次循环的时候需要取得最大值
    return ans


arr = [[0,0,1,0,0,0,0,1,0,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,0,0,0],
[0,1,1,0,1,0,0,0,0,0,0,0,0],
[0,1,0,0,1,1,0,0,1,0,1,0,0],
[0,1,0,0,1,1,0,0,1,1,1,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,0,1,1,1,0,0,0],
[0,0,0,0,0,0,0,1,1,0,0,0,0]]
process(arr)

## 问题2：[省份数量](https://leetcode-cn.com/problems/number-of-provinces/)
给定一个二维的0-1矩阵，如果第(i, j)位置是1,则表示第i个人和第j个人是朋友。已知朋友关系是可以传递的(即如果a是b的朋友，b是c的朋友，那么a和c也是朋友，换言之这三个人处于同一个朋友圈之内)。求一共有多少个朋友圈。

分析：
- 依旧使用主函数遍历+辅函数递归
- 递归函数中当i行的元素均为0，直接返回0，否则循环i行的每个元素将arr\[i]\[j] = 0 ，退出循环后返回1
- 主函数调用则是直接遍历行，然后计数即可

改进：
- 在调用的时候不必修改0，而时将搜索到的朋友放进visited集合中，使得其不必再被搜索
- 而dfs的作用添加修改visited,和递归调用
- 主函数则依旧是循环所有行，不过在循环的过程中跳过visited集合中已经存在的元素，只要是visited集合中不存在的即说明新的朋友圈被找到了

In [ ]:
def dfs(i, arr):
    if sum(arr[i]) == 0:
        return 0
    arr[i][i] = 0
    for j in range(len(arr[i])):
        if arr[i][j] == 1:
            arr[i][j] = 0
            arr[j][i] = 0
            dfs(j, arr)
    return 1

def process(arr):
    ans = 0
    for i in range(len(arr)):
        ans += dfs(i, arr)
    return ans

arr = [[1,1,0],[1,1,0],[0,0,1]]
process(arr)

In [ ]:
def dfs(i, arr, visited):
    visited.add(i)
    for j in range(len(arr)):
        if arr[i][j] == 1 and j not in visited:
            visited.add(j)
            dfs(j, arr, visited)

def process(arr):
    ans = 0
    visited = set()
    N = len(arr)
    for i in range(N):
        if i not in visited:
            ans += 1
            dfs(i, arr, visited)
    return ans
arr = [[1,0,0,1],[0,1,1,0],[0,1,1,1],[1,0,1,1]]
process(arr)

## 问题3：[太平洋大西洋水流问题](https://leetcode-cn.com/problems/pacific-atlantic-water-flow/)
给定一个二维的非负整数矩阵，每个位置的值表示海拔高度。假设左边和上边是太平洋，右边和下边是大西洋，求从哪些位置向下流水，可以流到太平洋和大西洋。水只能从海拔高的位置流到海拔低或相同的位置。

分析：
- 遍历数组中每一个位置来判断是否符合条件复杂度太高，我们可以考虑从海洋逆推回去，水流流向高处，再遍历一遍数组找到符合条件的位置。
- 可以利用两个集合储存符合条件的数组，取交集就可以。
- dfs函数输入给定一个坐标判断能否向上移动或者能否向周围移动，如果可以则放入到list中

改进：
- 由于每次查找元素是否在一个list中时间复杂度很高，因此我们可以用一个“二维数组取数字”来代替查找
- 注意：列表无法set

启发：
- 在写递归函数的时候如果是先操纵再递归，那么在主函数里面也应当先把第一个操作了再进入递归。如果是在递归开始再操作，那么在主函数里就直接调用即可

In [ ]:
def process(arr):
    def dfs(x, y, ocean):
        if 0 <= x-1 < n and arr[x][y] <= arr[x-1][y] and ([x-1,y] not in ocean):
            ocean.append([x-1, y])
            dfs(x-1, y, ocean)
        if 0 <= x+1 < n and arr[x][y] <= arr[x+1][y] and ([x+1,y] not in ocean):
            ocean.append([x+1, y])
            dfs(x+1, y, ocean)
        if 0<= y-1 < m and arr[x][y] <= arr[x][y-1] and ([x,y-1] not in ocean):
            ocean.append([x, y-1])
            dfs(x, y-1, ocean)
        if 0<= y+1 < m and arr[x][y] <= arr[x][y+1] and ([x,y+1] not in ocean):
            ocean.append([x, y+1])
            dfs(x, y+1, ocean)

    n = len(arr)
    m = len(arr[0])
    A = list()
    P = list()
    for i in range(n):
        A.append([i, 0])
        P.append([i, m-1])
    for j in range(m):
        if [0,j] not in A: A.append([0, j])
        if [n-1,j] not in P: P.append([n-1, j])

    for i in range(n):
        dfs(i, 0, A)
        dfs(i, m-1, P)
    for j in range(m):
        dfs(0, j, A)
        dfs(n-1, j, P)
    return [i for i in A if i in P]
    
arr = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
process(arr)

In [ ]:
def process(arr):
    def dfs(x, y, ocean):
        ocean[x][y] = 1
        if 0 <= x-1 < n and arr[x][y] <= arr[x-1][y] and not ocean[x-1][y] :
            dfs(x-1, y, ocean)
        if 0 <= x+1 < n and arr[x][y] <= arr[x+1][y] and not ocean[x+1][y]:
            dfs(x+1, y, ocean)
        if 0<= y-1 < m and arr[x][y] <= arr[x][y-1] and not ocean[x][y-1]:
            dfs(x, y-1, ocean)
        if 0<= y+1 < m and arr[x][y] <= arr[x][y+1] and not ocean[x][y+1]:
            dfs(x, y+1, ocean)

    n = len(arr)
    m = len(arr[0])
    A = [[0 for i in range(m)] for j in range(n)]
    P = [[0 for i in range(m)] for j in range(n)]

    for i in range(n):
        dfs(i, 0, A)
        dfs(i, m-1, P)
    for j in range(m):
        dfs(0, j, A)
        dfs(n-1, j, P)

    return [[i, j] for i in range(n) for j in range(m) if A[i][j] and P[i][j]]

arr = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
process(arr)

## 问题4:  [到达首都的最少油耗](https://leetcode.cn/problems/minimum-fuel-cost-to-report-to-the-capital)
给你一棵 n 个节点的树（一个无向、连通、无环图），每个节点表示一个城市，编号从 0 到 n - 1 ，且恰好有 n - 1 条路。0 是首都。给你一个二维整数数组 roads ，其中 roads[i] = [ai, bi] ，表示城市 ai 和 bi 之间有一条 双向路 。每个城市都有一个代表, 每座城市里有一辆车。给你一个整数 seats 表示每辆车里面座位的数目。相邻城市之间一辆车的油耗是一升汽油。问: 所有代表到达根节点需要消耗的油耗。

分析：
- 使用"贡献法"统计每一条边所贡献的油量。比如有一条边之后连着有5个节点，而每辆车只有2位置，那说明一共有ceil(5/2)=3辆车经过这条边，说明这条边一定会"贡献"3升油量。因此我们只需要计算每条边所贡献的油量，并求和即可
- 而每条边的油量，取决于该条边链接的节点有多少个，因此这个问题变成了统计每条边的子节点数，又可以理解为统计每个节点的每个子树上的节点数，因此我们使用dfs(深度优先搜索), 统计每条子树上的子节点个数
- 该题目中所有的末端叶节点，一定是出度为1的，因此一定只有一个邻居, 这个邻居就是他的父节点。因此我们在dfp中可以根据邻居是否唯一为父节点进行处理。
- 由于统计子节点是时还需要将计算贡献度, 我们可以使用一个全局变量进行累加贡献度(也可以单独使用数组记录下所有边的贡献度)

In [2]:
import math

neighbor_map = [[1,4,5],
                [0,3],
                [3],
                [1,2],
                [0,6],
                [0],
                [4]]   ## 邻居图

ans = 0   # 最后的答案
seats = 2 # 每辆车载重

def dfs(son, father):
    """返回给父节点，告知当前子树所有的节点个数
    """
    neighbors = neighbor_map[son]
 
    son_count = 1        # 当前子树的节点个数需要把自己加上
    for i in neighbors:  ## 遍历每个邻居
        if i != father:  ## 说明有子树(邻居不是father节点)
            son_count += dfs(i, son)
    
    ## 计算完当前father——son这条边的所有子节点个数后, 累加贡献值
    global ans
    ans += math.ceil(son_count/seats)

    return son_count


def process(neighbor_map):
    for i in neighbor_map[0]:
        dfs(i, 0)   # 遍历0节点的所有子树，并完成统计
    return ans


process(neighbor_map)

7

## 问题5:  [重新规划路线](https://leetcode.cn/problems/reorder-routes-to-make-all-paths-lead-to-the-city-zero/)
路线用 connections 表示，其中 connections[i] = [a, b] 表示从城市 a 到 b 的一条有向路线。n 座城市，从 0 到 n-1 编号，其间共有 n-1 条路线(两个城市间只有唯一一条路)。请你帮助重新规划路线方向，使每个城市都可以访问城市 0 。返回需要变更方向的最小路线数。

分析：
- 由于每个城市都要到达0, 因此可以从0出发, 遍历与0连接的所有边,如果是[其他点, 0]则无需改变方向，如果是[0, 其他点]则需要改变方向。下一轮我们只需要遍历“其他点”的边，使得这些边都以“其他点”作为终点
- 使用深度优先遍历, 从0节点开始遍历与0相连的所有边, 并判定这些边是否符合条件(即终点为0), 若不符合条件, 则ans+=1。接着遍历下一个节点的边

In [12]:
def dfs(current, parent): 
    global edges
    global ans   ## 用全局变量储存
    for start, end in edges[current]:  ## 遍历current的每一条边, start表示起点, end表示终点
        if start == parent or end == parent:   ## 如果边中涉及到了父节点, 那么不用处理
            continue
        if end != current:  ## 如果终点不为当前点, 则计数+1, 说明要转换方向
            ans += 1
            dfs(end, current)  ## 此时current为父节点, end为下一个需要搜索的节点
        else:
            dfs(start, current) ## 此时current为父节点, start为下一个需要搜索的节点


### 以下是题目入参
n = 6
connections = [[0,1],[1,3],[2,3],[4,0],[4,5]]
ans = 0

### 以下是解决
edges = [[] for _ in range(n)]
for a, b in connections:
    edges[a].append([a,b])
    edges[b].append([a,b])
dfs(0, -1)

print(ans)

[[0, 1], [4, 0]]
3


# 回溯法
在搜索到某一节点的时候，如果我们发现目前的节点（及其子节点）并不是需求目标时，我们回退到原来的节点继续搜索，并且把在目前节点修改的状态还原，只是多了回溯的步骤，变成了\[修改当前节点状态]——>\[递归子节点]——>\[回改当前节点状态]
- 小诀窍:一是按引用传状态，二是所有的状态修改在递归完成后回改。
- 回溯法修改一般有两种情况:一种是修改最后一位输出，比如排列组合；一种是修改访问标记，比如矩阵里搜字符串

## 问题4：[全排列](https://leetcode-cn.com/problems/permutations/)
给定一个无重复数字的整数数组，求其所有的排列方式。例:\[1,2,3],输出为：\[\[1,2,3], \[1,3,2], \[2,1,3], \[2,3,1], \[3,1,2], \[3,2,1]]

分析:
- 全排列可以使用循环互换来进行：第1个数与1,2,3..。互换，互换之后递归调用第2个数与2.3.4...互换(可以看成第i位置有多少种数字的可能)，当进行到与最后一个元素交换时停止。(这里的交换可以看成是在每个位置上放数字,只不过交换的这种方式不需要额外的储存变量)
- 注意在每次递归之后需要还原(因为第一个个数与第2个数交换了之后，不能影响到第一个数与第3个数交换)

In [ ]:
def process(arr, level, ans):
    if level >= len(arr)-1:
        ans.append(arr.copy())
        return
    for i in range(level, len(arr)):
        arr[i], arr[level] = arr[level], arr[i]
        process(arr, level+1, ans)
        arr[i], arr[level] = arr[level], arr[i]
ans = []
process([1,2,3], 0, ans)
ans

## 问题5：[组合](https://leetcode-cn.com/problems/combinations/)
给定一个整数n和一个整数k，求在1到n中选取k个数字的所有组合方法。例：n = 4, k = 2，返回\[\[2,4],\[3,4],\[2,3],\[1,2],\[1,3],\[1,4]]

分析：
- 排列回溯的是交换的位置，而组合回溯的是否把当前的数字加入结果中(和排列不一样的是组合是从一个待选集合里选择是否放入其中一个)
- 利用一个空数组，每次传入一个数，数组数等于k时返回。k表示剩余有多少数字需要选，begin表示从数字几开始选(备选集合)。
- 剪枝：举个例子如果我们从7开始选(最多选10)，还需要选5个数，那么我们可以直接返回了，因为剩下的数不够我们选了，因此剪枝条件是：begin > n-(k-1)

In [ ]:
def backtrack(n, begin, k, arr):
    if begin > n-k+1:   #剪枝条件
        return
    if k == 0:
        global ans
        ans.append(arr.copy())
        return 
    for i in range(begin, n+1):
        arr.append(i)
        backtrack(n, i+1, k-1, arr)
        arr.pop()
n = 4
k = 2
ans = []
backtrack(n, 1, k, [])
ans

## 问题6：[单词搜索](https://leetcode-cn.com/problems/word-search/)
给定一个字母矩阵，所有的字母都与上下左右四个方向上的字母相连。给定一个字符串，求字符串能不能在字母矩阵中寻找到。
例:board = \[\["A","B","C","E"],\["S","F","C","S"],\["A","D","E","E"]], word = "ABCCED", 返回true

分析：
- 在递归中为了防止向左移动以后，又向右移动，我们需要一个二维矩阵来储存我们是否访问过节点
- 为了加速，我们可以先遍历一遍数组找到开头匹配的字母以及数字


In [ ]:
def backtrack(x, y, board, word, visited, pos):
    # 超出边界处理&节点被访问过
    if x>=len(board) or y>=len(board[0]) or x<0 or y<0 or visited[x][y]:
        return False

    # 该位置是否满足条件
    if word[pos] != board[x][y]:
        return False

    # 终止递归的条件(先判断已经判断了是否符合条件，所以这里只用判断是否为最后一个即可)
    if pos == len(word)-1:
        return True

    visited[x][y] = 1   # 该位置标记访问
    find = (backtrack(x+1, y, board, word, visited, pos+1) 
           or backtrack(x, y+1, board, word, visited, pos+1) 
           or backtrack(x-1, y, board, word, visited, pos+1)
           or backtrack(x, y-1, board, word, visited, pos+1))
    visited[x][y] = 0   # 恢复状态
    return find

def main(board, word):
    n = len(board)
    m = len(board[0])
    visited = [[0]*m for i in range(n)]
    # 遍历一遍board，收集到开头匹配的位置
    can = []
    for i in range(n):
        for j in range(m):
            if board[i][j] == word[0]:
                can.append((i, j))
    for i, j in can:
        if backtrack(i, j, board, word, visited, 0):
            return True
    return False

board =[["s",'a']]
word = "a"
main(board, word)


## 问题7：[N 皇后](https://leetcode-cn.com/problems/n-queens/)
在N*N的棋盘上要摆放N个皇后，要求任何两个皇后不同行、不同列，也(2个)不再一条斜线上。问n皇后的摆法有多少种？(皇后是一样的，但是位置不同算一种))

分析：
- 由于要在N*N中摆放N个皇后因此，每一行必须也只能有1个皇后,因此可以构建一维数组来表示每一行的某一列是否摆放皇后
- 可以从第一行开始"试",如果第一个皇后摆第一个位置,那么第二个皇后有几种可能？并且每种可能会导致第三行皇后有几种可能？知道出现第i行的皇后有0种可能的时候,回退到第i-1行试下一种可能。直到第一行的皇后都试完了
- 采用数组记录每一行的皇后的坐标(其实只需要记录"列")
- 共斜线的判定法则是|X1-X2|==|Y1-Y2|是否成立(横纵坐标差的绝对值是否相等。)
    - 共斜线还有一种判定是：x1+y1 == x2+y2 or x1-y1 == x2-y2
- 注意：将一个列表加入一个列表中的的时候要注意深浅复制


In [1]:
def isSuitable(arr, N, j):
    '''
        arr: 最后的摆放结果
        N: 当前摆放第N+1个皇后<=>摆放第N+1行的那个皇后
        j: 在第j列摆放
    '''
    for i in range(N):  # 搜寻前N个皇后的位置信息
        if arr[i] == j or abs(i-N) == abs(arr[i]-j):
            return False
    return True

def backtrack(arr, N, ans):
    '''
        N: 当前摆放第N+1个皇后,摆放第N+1行的那个皇后
        ans: 用来储存最后摆放位置结果, arr[i] = j表示第i+1个皇后摆放在第(i,j)
    '''
    if N == len(arr):
        ans.append(arr.copy())
        return ans
    for j in range(len(arr)):
        ## 当位置1可以进入后, 位置2也会能重新进入循环中判断
        if isSuitable(arr, N, j):
            arr[N] = j
            backtrack(arr, N+1, ans)
            ## 此处不用回溯也不会影响后续的判断
    return ans
        
arr = [-1]*4
ans = []
backtrack(arr, 0, ans)


[[1, 3, 0, 2], [2, 0, 3, 1]]

# 广度优先搜索
广度优先搜索（breadth-first search，BFS）不同与深度优先搜索，它是一层层进行遍历的，因此需要用先入先出的队列而非先入后出的栈进行遍历。由于是按层次进行遍历，广度优先搜索时按照“广”的方向进行遍历的，也常常用来处理最短路径等问题。

深度优先搜索和广度优先搜索都可以处理可达性问题，但是而用栈实现的深度优先搜索和用队列实现的广度优先搜索在写法上并没有太大差异，因此使用哪一种搜索方式需要根据实际的功能需求来判断。

## 问题8：[最短的桥](https://leetcode-cn.com/problems/shortest-bridge/)
给定一个二维 0-1 矩阵，其中1表示陆地，0表示海洋，每个位置与上下左右相连。已知矩阵中有且只有两个岛屿，求最少要填海造陆多少个位置才可以将两个岛屿相连.

分析
- 深度遍历和广度遍历都需要，先使用普通循环找到第一个岛的登录地，开始使用深度遍历遍历完的第一个岛遍历过程中用队列queue收集与第一个岛接壤的所有海水。
- 在完成上一步之后可以得到与第一个岛所有接壤的海水点队列queue，然后开始广度遍历(相当于探索一遍queue元素的所有点，看与他们接触的有没有1，如果探索完一遍没有1则探索第二层)，在探索queue的所有点i时，i周围如果不是1，则需要把i周围的点放到第二层队列里。
- 无论是深度遍历还是广度遍历，在遍历过的点都需要记录下(也就是沉掉)，防止在遍历左边的时候又遍历回来右边了。。。

In [ ]:
grid = [[0,1,0],[0,0,0],[0,0,1]]
n = len(grid)
m = len(grid[0])
queue = []

def dfs(x, y):
    if x<0 or x>=n or y<0 or y>=m or grid[x][y] == 2:
        return
    if grid[x][y] == 0:
        global queue
        queue.append((x, y))  # 收集岛屿周围的水域坐标
        return
    grid[x][y] = 2 #把遍历到的第一个岛的所有点沉为2
    dfs(x-1, y)
    dfs(x+1, y)
    dfs(x, y+1)
    dfs(x, y-1)

# 每一层的广度遍历
def bsf(queque):
    count = len(queque)   # 记录本层应该遍历多少次这个点
    while count:
        x, y = queque.pop(0)
        count -= 1
        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
            if 0<=x+dx<n and 0<=y+dy<m and grid[x+dx][y+dy] != 2: 
                if grid[x+dx][y+dy] == 1:   # 说明碰到了第二个岛，返回
                    return True
                queue.append((x+dx, y+dy))  # 加入到队列尾部
                grid[x+dx][y+dy] = 2        # 标记一下，一会再遍历到他的时候跳过
    return False

def main():
    find = False    # 用来提前结束循环
    for i in range(n):
        if find : break
        for j in range(m):
            if grid[i][j] == 1: #说明找到了第一个岛
                dfs(i, j)    ## 深度优先搜索, 收集岛屿周围的水域坐标
                find = True
                break
    # 此时已经获得了第一个岛的周围水域点queue
    level = 0
    global queue
    while queue:
        level += 1
        if bsf(queue):
            return level
main()


## 问题9-1: [单词接龙 I](https://leetcode.cn/problems/word-ladder/)
### 题目
给定一个起始字符串beginWord和一个终止字符串endWord，以及一个单词表List，求是否可以将起始字符串每次改一个字符，直到改成终止字符串，且所有中间的修改过程表示的字符串都可以在单词表里找到。如存在则输出需要转化路径的长度，若不存在这样的路径则输出0。

例如：beginWord = "hit", endWord = "cog", wordList = \["hot","dot","dog","lot","log","cog"], 我们可以通过"hit" -> "hot" -> "dot" -> "dog" -> "cog"这样的转换方式来得到“cog”，也可以用"hit" -> "hot" -> "lot" -> "log" -> "cog"来得到。可以看到转换路径中所有的单词都能在wordList中找到。因此我们输出最短的路径长度为:5

### 分析
- 先写一个函数getNeighbors用来获取指定单词的邻居(子结点)有那些。也就是在做邻接矩阵，返回一个字典，key为节点，value为一个列表表示与key相连的单词有那些。
- 拿到邻接矩阵之后我们我们就可以做BFS(广度优先搜索)，找到从beginWord到endWord的最短路径了。
- BFS的思路是:
    - 从beginWord开始，采用广度遍历基本操作——队列，把遍历到的邻居的邻居放入队列等待下一次遍历。
    - 如果遍历到了endWord,则可以停止之后的邻居的遍历了，但要注意把endWord那一层队列全部遍历完

### 感悟
- 该题由于邻接矩阵中所有的边都是等权重的，因此该题的解题方法也可以应用在所有的等权的路径图(有向无向都可以吧？)中
- 若邻接矩阵中的边是不等权的，则我们需要使用Dijkstra算法(14章中有介绍)来得到最短路径。

In [4]:
def getNeighborMap(beginWord, wordList):
    '''
    word: str
    wordList: ["word1", "word2", ...]
    return: {str:[邻居..], str2:[邻居....]}
    '''
    wordSet = set(wordList)
    wordSet.add(beginWord)
    result = dict()
    for word in wordSet:
        word = list(word)
        n = len(word)
        neighbors = []
        # 改变word的每个字母
        for i in range(n):
            old = word[i]
            for j in range(97, 123):
                if chr(j) == old:   # 如果和自己字母一样的话需要跳过
                    continue
                word[i] = chr(j)    # 改变第i号位置的字母
                new_word = ''.join(word) 
                if new_word in wordSet:  # 新字母如果在单词表中说明这俩能值相差一个字母
                    neighbors.append(new_word)
                word[i] = old  # 别忘了改回原来的字母
        result[''.join(word)] = neighbors
    return result


def bfs(beginWord, endWord, wordList):
    """
    return:
        {'hit': 1, 'hot': 2, 'dot': 3, 'lot': 3, 'dog': 4, 'log': 4, 'cog': 5}
    """
    graph = getNeighborMap(beginWord, wordList)
    queue = []
    queue.append(beginWord)
    depth = 0
    result = dict()  # 返回的结果
    isFind = False   # endWord是否被找到
    while queue:
        neighbor_num = len(queue)   # 该层的所有邻居数量，也就pop的次数
        depth += 1
        for _ in range(neighbor_num):
            temp = queue.pop(0)             # 拿出节点
            if temp in result:              # 如果该节点已经被记录了则跳过
                continue
            if temp == endWord:             # 说明已经遍历到了终点,可以不用再遍历队列了,但是当前层的所有节点需要遍历完
                isFind = True
            result[temp] = depth   ## 记录距离
            next_neighbors = graph.get(temp, []) # 获得节点的邻居列表
            for item in next_neighbors:
                if item not in result:   # 如果这个节点没有被遍历过，则加入队列
                    queue.append(item)
        if isFind:
            break
    return result
    
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
result = bfs(beginWord, endWord, wordList)
result.get(endWord,0)


5

## 问题9-2: [单词接龙 II](https://leetcode-cn.com/problems/word-ladder-ii/)
### 题目
要求与问题9(单词接龙 I)一样，只不过要求输出最短路径的路径列表，例如beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]，该题有两条路径可以使得beginWord ——> endWord，因此输出列表为：[["hit","hot","dot","dog","cog"],["hit","hot","lot","log","cog"]]


### 分析
- 我们在第一问中已经获得了从beginWord出发到endWord的最短路径，并且还获得了中间节点的距离。我们可以根据这个距离字典来进行DFS深度遍历，获得最后结果。一位内深度遍历情况下一定能经过从beginWord出发到endWord的最短路径。(并且还会记录下多种情况)
- DFS的写法
    - 终止条件是如果，起始单词与终止单词是一样的，意味着已经达到终点了，收集答案，并返回。
    - 在向下探索节点时我们使用了distance\[item] == distance\[beginword]+1的判定法则来进行”剪枝“,因为distance记录的就是各个节点最近距离的的层数，如果不满足这个条件则意味着有更近的路径因此我们会抛弃这一条路。


In [ ]:
def dfs(beginWord, endWord, distance, neighborMap, tem, ans):
    '''
    tmp: 是记录递归时候路径,走到最后将会把答案储存在ans中
    distance: {'hit':1, 'hot':2,...}表示从beginWord出发到各节点的最短距离
    neighborMap: {str:[邻居..], str2:[邻居....]}
    '''
    if beginWord == endWord:    # 当起始位置和终止位置一致的时候停止递归
        ans.append(tem.copy())
        return ans
    neighbors = neighborMap.get(beginWord)
    for item in neighbors:
        if distance.get(item, -1) == distance[beginWord] + 1:   # 如果在distance中item层数不是beginWord层数的下一层，意味着item不在我们“需要”的路径里
            tem.append(item)
            dfs(item, endWord, distance, neighborMap, tem, ans)
            tem.pop()   # 回溯法复原
    return ans


def process(beginWord, endWord, wordList):
    ans = []
    wordSet = set(wordList)
    if endWord not in wordSet:
        return ans
    distance = bfs(beginWord, endWord, wordList)      ## 广度搜索获得距离
    neighborMap = getNeighborMap(beginWord, wordList) ## 获得邻接矩阵

    # print(neighborMap)
    # print(distance)
    
    temp_ans = [beginWord]
    dfs(beginWord, endWord, distance, neighborMap, temp_ans, ans)
    return ans


beginWord = "aaaaa"
endWord = "ggggg"
wordList = ["aaaaa","caaaa","cbaaa","daaaa","dbaaa","eaaaa","ebaaa","faaaa","fbaaa","gaaaa","gbaaa","haaaa","hbaaa","iaaaa","ibaaa","jaaaa","jbaaa","kaaaa","kbaaa","laaaa","lbaaa","maaaa","mbaaa","naaaa","nbaaa","oaaaa","obaaa","paaaa","pbaaa","bbaaa","bbcaa","bbcba","bbdaa","bbdba","bbeaa","bbeba","bbfaa","bbfba","bbgaa","bbgba","bbhaa","bbhba","bbiaa","bbiba","bbjaa","bbjba","bbkaa","bbkba","bblaa","bblba","bbmaa","bbmba","bbnaa","bbnba","bboaa","bboba","bbpaa","bbpba","bbbba","abbba","acbba","dbbba","dcbba","ebbba","ecbba","fbbba","fcbba","gbbba","gcbba","hbbba","hcbba","ibbba","icbba","jbbba","jcbba","kbbba","kcbba","lbbba","lcbba","mbbba","mcbba","nbbba","ncbba","obbba","ocbba","pbbba","pcbba","ccbba","ccaba","ccaca","ccdba","ccdca","cceba","cceca","ccfba","ccfca","ccgba","ccgca","cchba","cchca","cciba","ccica","ccjba","ccjca","cckba","cckca","cclba","cclca","ccmba","ccmca","ccnba","ccnca","ccoba","ccoca","ccpba","ccpca","cccca","accca","adcca","bccca","bdcca","eccca","edcca","fccca","fdcca","gccca","gdcca","hccca","hdcca","iccca","idcca","jccca","jdcca","kccca","kdcca","lccca","ldcca","mccca","mdcca","nccca","ndcca","occca","odcca","pccca","pdcca","ddcca","ddaca","ddada","ddbca","ddbda","ddeca","ddeda","ddfca","ddfda","ddgca","ddgda","ddhca","ddhda","ddica","ddida","ddjca","ddjda","ddkca","ddkda","ddlca","ddlda","ddmca","ddmda","ddnca","ddnda","ddoca","ddoda","ddpca","ddpda","dddda","addda","aedda","bddda","bedda","cddda","cedda","fddda","fedda","gddda","gedda","hddda","hedda","iddda","iedda","jddda","jedda","kddda","kedda","lddda","ledda","mddda","medda","nddda","nedda","oddda","oedda","pddda","pedda","eedda","eeada","eeaea","eebda","eebea","eecda","eecea","eefda","eefea","eegda","eegea","eehda","eehea","eeida","eeiea","eejda","eejea","eekda","eekea","eelda","eelea","eemda","eemea","eenda","eenea","eeoda","eeoea","eepda","eepea","eeeea","ggggg","agggg","ahggg","bgggg","bhggg","cgggg","chggg","dgggg","dhggg","egggg","ehggg","fgggg","fhggg","igggg","ihggg","jgggg","jhggg","kgggg","khggg","lgggg","lhggg","mgggg","mhggg","ngggg","nhggg","ogggg","ohggg","pgggg","phggg","hhggg","hhagg","hhahg","hhbgg","hhbhg","hhcgg","hhchg","hhdgg","hhdhg","hhegg","hhehg","hhfgg","hhfhg","hhigg","hhihg","hhjgg","hhjhg","hhkgg","hhkhg","hhlgg","hhlhg","hhmgg","hhmhg","hhngg","hhnhg","hhogg","hhohg","hhpgg","hhphg","hhhhg","ahhhg","aihhg","bhhhg","bihhg","chhhg","cihhg","dhhhg","dihhg","ehhhg","eihhg","fhhhg","fihhg","ghhhg","gihhg","jhhhg","jihhg","khhhg","kihhg","lhhhg","lihhg","mhhhg","mihhg","nhhhg","nihhg","ohhhg","oihhg","phhhg","pihhg","iihhg","iiahg","iiaig","iibhg","iibig","iichg","iicig","iidhg","iidig","iiehg","iieig","iifhg","iifig","iighg","iigig","iijhg","iijig","iikhg","iikig","iilhg","iilig","iimhg","iimig","iinhg","iinig","iiohg","iioig","iiphg","iipig","iiiig","aiiig","ajiig","biiig","bjiig","ciiig","cjiig","diiig","djiig","eiiig","ejiig","fiiig","fjiig","giiig","gjiig","hiiig","hjiig","kiiig","kjiig","liiig","ljiig","miiig","mjiig","niiig","njiig","oiiig","ojiig","piiig","pjiig","jjiig","jjaig","jjajg","jjbig","jjbjg","jjcig","jjcjg","jjdig","jjdjg","jjeig","jjejg","jjfig","jjfjg","jjgig","jjgjg","jjhig","jjhjg","jjkig","jjkjg","jjlig","jjljg","jjmig","jjmjg","jjnig","jjnjg","jjoig","jjojg","jjpig","jjpjg","jjjjg","ajjjg","akjjg","bjjjg","bkjjg","cjjjg","ckjjg","djjjg","dkjjg","ejjjg","ekjjg","fjjjg","fkjjg","gjjjg","gkjjg","hjjjg","hkjjg","ijjjg","ikjjg","ljjjg","lkjjg","mjjjg","mkjjg","njjjg","nkjjg","ojjjg","okjjg","pjjjg","pkjjg","kkjjg","kkajg","kkakg","kkbjg","kkbkg","kkcjg","kkckg","kkdjg","kkdkg","kkejg","kkekg","kkfjg","kkfkg","kkgjg","kkgkg","kkhjg","kkhkg","kkijg","kkikg","kkljg","kklkg","kkmjg","kkmkg","kknjg","kknkg","kkojg","kkokg","kkpjg","kkpkg","kkkkg","ggggx","gggxx","ggxxx","gxxxx","xxxxx","xxxxy","xxxyy","xxyyy","xyyyy","yyyyy","yyyyw","yyyww","yywww","ywwww","wwwww","wwvww","wvvww","vvvww","vvvwz","avvwz","aavwz","aaawz","aaaaz"]

# process(beginWord, endWord, wordList)
getNeighborMap(beginWord, wordList)


## 问题10: [被围绕的区域](https://leetcode-cn.com/problems/surrounded-regions/)
给你一个mxn的矩阵board，由若干字符'X'和'O'，找到所有被'X'围绕的区域，并将这些区域里所有的‘O'用'X'填充。

分析：
- 这道题题目最难的地方是如何识别出与边界相连的O，如果我们把与边界相连的O记录下来，那么剩下的O一定是被X完全包裹的，所以我们识别出与边界相连的O之后，可以再次遍历一遍数组把那些O变成X就可以了
- 如何识别出与边界相连的O有多种代码实现，下面将展示两种分别是基于递归DFS，和非递归BFS的写法

In [7]:
def dfs(board, i ,j):
    '''
    将'O'的位置转化为'#',从边缘开始搜索,将边缘的'O'全部更换成'#', 即这些'O'是不需要进行转化的'O'
    '''
    if i<0 or j<0 or i>=len(board) or j>=len(board[0]) or board[i][j] != 'O':
        return
    board[i][j] = '#'   # 标记遍历过的点
    dfs(board, i+1 ,j)
    dfs(board, i-1 ,j)
    dfs(board, i ,j+1)
    dfs(board, i ,j-1)


def bfs(board, i, j):
    if i<0 or j<0 or i>=len(board) or j>=len(board[0]) or board[i][j] != 'O':
        return
    queue = [(i,j)]
    board[i][j] = '#'
    while queue:
        x, y = queue.pop()
        for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            if 0<=x+dx<len(board) and 0<=y+dy<len(board[0]) and board[x+dx][y+dy] == 'O':
                queue.append((x+dx, y+dy))
                board[x+dx][y+dy] = '#'
    
def process(board):
    n = len(board)
    m = len(board[0])
    ## 从边缘开始搜索，将边缘开始的所有'O'块换成'#'
    for i in range(n):
        bfs(board, i, 0)    # 可以把bfs换成dfs
        bfs(board, i, m-1)  
    for j in range(m):
        bfs(board, 0, j)   
        bfs(board, n-1, j)  
    # 上述代码已经与边缘相接的所有O快换成了#，接下来只需要遍历，将剩下的'O'转化为'X'即可

    for i in range(n):
        for j in range(m):
            if board[i][j] == "#":
                board[i][j] = 'O'
            elif board[i][j] == "O":
                board[i][j] = "X"

board = [["X","X","X","X"],
         ["X","O","O","X"],
         ["X","X","O","X"],
         ["X","O","X","X"]]
process(board)
board


更改后: [['X', 'X', 'X', 'X'], ['X', 'O', 'O', 'X'], ['X', 'X', 'O', 'X'], ['X', '#', 'X', 'X']]


[['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'O', 'X', 'X']]

## 问题11:[二叉树的所有路径](https://leetcode-cn.com/problems/binary-tree-paths/)
给定一个二叉树的根节点，返回所有从根节点到叶子节点的路径。输出：\["1->2->5", "1->3"]，节点属性：self.val = val。self.left = left。self.right = right

分析：
- 递归打印回溯即可

In [ ]:
def process(root, path, ans):
    '''
    将root节点加入path路径
    '''
    if root.left == None and root.right == None:
        path = path + str(root.val)
        ans.append(path)
        return ans
    
    old = path   ## 临时变量储存一下旧路径。
    if root.left != None:
        path = path + str(root.val) + "->"
        process(root.left, path, ans)
        path = old
    if root.right != None:
        path = path + str(root.val) + "->"
        process(root.right, path, ans)
        path = old


## 问题12: [全排列 II](https://leetcode-cn.com/problems/permutations-ii/)
给定一个可包含重复数字的序列nums ，按任意顺序返回所有不重复的全排列。

分析：
- 全排列，交换次序即可(只跟后面的次序换)，不过记得要换回来。(起始可以看成在第i个位置有那些情况)
- 由于存在重复的数字，因此我们在换的时候需要剪枝, 我们需要记录在同一个位置不能出现两次两次相同的互换位置，因此在每一个位置可以用一个集合来储存还没使用过的数字

In [1]:
arr = [1,1,2]
n = len(arr)
def process(arr, index):
    if index >= n-1:
        ans.append(arr.copy())
        return ans
    unused = set(arr)  # 每一层都要
    for i in range(index, n):
        # 这个循环里面都表示在index位置的数值的所有情形
        if arr[i] in unused:
            unused.remove(arr[i])
            arr[index], arr[i] = arr[i], arr[index]
            process(arr, index+1)
            arr[index], arr[i] = arr[i], arr[index]
    return ans
ans = []
process(arr, 0)


[[1, 1, 2], [1, 2, 1], [2, 1, 1]]

## 问题13: [组合总和 II](https://leetcode-cn.com/problems/combination-sum-ii/)
给定一个数组 candidates 和一个目标数 target ，找出 candidates 中所有可以使数字和为 target 的组合。例如：candidates = \[10,1,2,7,6,1,5], target = 8,则解的集合为：\[\[1, 7],\[1, 2, 5],\[2, 6],\[1, 1, 6]]

分析：
- 由于数字只出现一次则采用组合数的方法进行，采用回溯法
- 这里要剔除重复集合，主要采用先将candidates排序，这样我们在每一层选取数字的时候如果遇到与本层相同的数字的时候就可以剪枝了(因为在如果选择相同的数字那下一层的备集合一定是刚才前面选过数字的下一层备选集的真子集)

In [ ]:
ans = []
def process(arr, tempans, target, k):
    '''
        从arr的k位置开始,需要选取数值和为target。
        arr:需要被排好序
    '''
    if target == 0:
        ans.append(tempans.copy())
        return 

    if k >= len(arr) or target<0:
        return
    
    used = set()
    for i in range(k, len(arr)):
        if arr[i] not in used:
            used.add(arr[i])
            target -= arr[i]
            tempans.append(arr[i])
            process(arr, tempans, target, i+1)
            ## 回溯
            target += arr[i]
            tempans.pop()

candidates = [10,1,2,7,6,1,5]
candidates.sort()
target = 8 
process(candidates, [], 8, 0)
ans

In [ ]:
ans = []
def process(arr, tempans, target, k):
    '''
        arr：需要被排好序
    '''
    if target == 0:
        ans.append(tempans[:])
        return 

    if k >= len(arr) or target<0:
        return
    
    for i in range(k, len(arr)):
        if i == k or arr[i] != arr[i-1]:
            target -= arr[i]
            tempans.append(arr[i])
            print(i, tempans)
            process(arr, tempans, target, i+1)
            target += arr[i]
            tempans.pop()

candidates = [10,1,2,7,6,1,5]
candidates.sort()
target = 8 
process(candidates, [], 8, 0)
ans

## 问题14: [解数独](https://leetcode-cn.com/problems/sudoku-solver/)
题目：
数独的解法需 遵循如下规则：数字 1-9 在每一行只能出现一次。数字 1-9 在每一列只能出现一次。数字 1-9 在每一个以粗实线分隔的 3x3 宫内只能出现一次。（请参考示例图）

分析：
- 主要采用回溯方，每个空白的地方都要去试一试。
- 难点在于怎么构造表示判断数字是否能填入该空格中的方法(同一行不能出现相同，同一列不能出现相同，同一个3*3方格不能出现相同)
    - 利用三个数组来分别表示行，列，方块中数字的使用情况
    - row[i][j] = Ture:表示第i行中使用了j这个数字。i：0~8,j:1~9
    - col[i][j] = Ture:表示第i列中使用了j这个数字。i：0~8,j:1~9
    - box[i][j][k] = Ture:表示第i行第j列个方格中使用了k这个数字。i：0~2,j:0~2,k:1~9(把9*9的小方块划分成3*3的大方块)
    - 更新：如果row[i][x]，col[j][x]，box[i//3][j//3][x]都为False，意味着第i行，第j列，可以填写x这个数字，但凡有一个是ture，都意味着x不能填在这里。
- 为了加快速度，可以在初始化row,col,box时储存下来需要填写的位置

In [ ]:
def is_suitable(i, j, x):
    return not (row[i][x] or col[j][x] or box[i//3][j//3][x])

# 递归函数，用全局变量valid来控制结束递归时间
def dfs(pos):
    '''
    spaces: [(i1,j1),(i2,j2)..] 表示储存空白位置坐标集合，该位置还没有数字
    '''
    global valid
    if pos == len(spaces):
        valid = True
        return 
    i, j = spaces[pos]  # 得到一个空白位置坐标
    for number in range(1,10):
        if is_suitable(i,j,number):
            row[i][number] = col[j][number] = box[i//3][j//3][number] = True
            board[i][j] = str(number)
            dfs(pos+1)
            #回溯
            row[i][number] = col[j][number] = box[i//3][j//3][number] = False 
        if valid:
            return


row = [[False]*10 for _ in range(9)]
col = [[False]*10 for _ in range(9)]
box = [[[False]*10 for _ in range(3)] for __ in range(3)]
spaces = []     # 储存空白位置，元素为(i,j)表示该位置还没有数字
valid = False   # 是否停止第归
board = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]

# 根据board初始化这三个bool数组, 并记录空白位置
for i in range(9):
    for j in range(9):
        if board[i][j] != '.':
            number = eval(board[i][j])
            row[i][number] = True
            col[j][number] = True
            box[i//3][j//3][number] = True
        else:
            spaces.append((i,j))    # 储存空白位置
dfs(0)
board

## 问题15: [最小树高度](https://leetcode-cn.com/problems/minimum-height-trees/)
题目：
这里的树指的是没有环的无向图，给你一棵包含n个节点的无向图，标记为0到n-1。给定数字n和一个有n-1条无向边的edges列表（每一个边都是一对标签），其中edges[i] = [ai, bi] 表示树中节点ai和bi之间存在一条无向边，可选择树中任何一个节点作为根。当选择节点x作为根节点时，设结果树的高度为h 。在所有可能的树中，具有最小高度的树（即，min(h)）被称为最小高度树。问题：找出最小高度树的根节点，返回列表。例如：n = 4, edges = [[1,0],[1,2],[1,3]]，节点0，1，2，3，都可以作为根结点，一共有4种情况，但是只有当1作为根结点时候，这棵树是最小高度的，因此返回根结点的【1】

分析：
- 先把相互之间的边构造成字典{number：[numbers],...}
- 采用类似于按层遍历二叉树的方式来记录每个节点作为根节点树的层树：
    - 构建队列，每次从首部弹出节点，并把对应的子节点放入对列尾部，记录下一层的next_endnumber
    - 每次弹出首节点的时候判度是否为endnumber,如果是则层数++，并把next_endnumber赋值给endnumber
    - 直到队列为空停止
- 这一题中由于没有环，在节点进入队列前需要判定一下是否已经被记录过了。
- 结果：超出时间限制

优化：
- 从入度为1的开始遍历，因为(大于两个节点的树)入度为1的数字作为根节点树的长度一定不是最短的，因此这些入度为1的点在最小树里面一定是最后一个子节点。从而我们可以删除这些入度为1的节点，再次往上遍历，再删除入度为1的节点
- 我们并不需要求出最后最小树的准确高度，我们只需要按照这种方法来删除就能确保最后剩下的1个或者2个入度为1的点一定是最小树的根结点。
- 根据题目我们知道最小树的根节点一定是1个或者2个，不可能是3个点，因为n个数字用n-1条边联结起来，按照上述的删除方法一定只剩下有1个或者2个点。


### 方法一：暴力遍历(超出时间限制)

In [ ]:
def process(n:int, edges):
    # 构造邻居字典
    neighbors = {i:[] for i in range(n)}
    for i,j in edges:
        neighbors[i].append(j)
        neighbors[j].append(i)

    min_layer = n   # 设置初始最小层树
    res = []        # 记录最小树的根结点

    # 计算每一个数字作为根节点的层数
    for root_number in range(n):
        layer = bfs(root_number, neighbors, n)
        if min_layer > layer:
            min_layer = layer
            res = []
            res.append(root_number)
        elif min_layer == layer:
            res.append(root_number)
    return res

# 用于计算给定根节点的树的层数
def bfs(root_number, neighbors, n):
    used = [True for _ in range(n)]     # 记录使用过的节点
    queue = [root_number]   # 把根节点放入队列
    used[root_number] = False   # 记录根结点被使用
    end_number = root_number    # 本层的终止节点
    next_end_number = None      # 下一层的终止节点
    layer = 0
    while queue:
        number = queue.pop(0)
        for nbor in neighbors[number]:
            if used[nbor]:
                queue.append(nbor)
                used[nbor] = False
                next_end_number = nbor
        if number == end_number:
            layer += 1
            end_number = next_end_number
    return layer

### 优化一

In [ ]:
def process(n, edges):
    if n == 1:
        return [0]
    elif n == 2:
        return [0,1]

    # 构造邻居字典
    neighbors = {i:[] for i in range(n)}
    for i,j in edges:
        neighbors[i].append(j)
        neighbors[j].append(i)
    
    queue = []      # 队列
    for key, value in neighbors.items():
        if len(value) == 1:  ## 将入度为1的点放入队列中
            queue.append(key)

    while queue:
        size = len(queue)   # 通过长度来控制层的遍历
        n = n - size        # n个节点即将被删掉size个，剩余的节点数
        for _ in range(size):        
            number = queue.pop(0)       # 拿出入度为1的点，接下来要将number从整个邻居字典中删除
            nbor = neighbors[number][0]     # 找到入度为1的点那个邻居（只有1个所以直接取出来就可以了）

            neighbors[nbor].remove(number)  # 在这个邻居的[]中删除这个number(相当于在删除这个number)
            if len(neighbors[nbor]) == 1:
                queue.append(nbor)      # 如果删除点之后，这个邻居也成了入读为1的，那么这个nbor也应该加入到队列中等待被删除

        if n == 1:
            return [queue.pop()]
        elif n == 2:
            return [queue.pop(),queue.pop()]

## 问题16: [二叉树的堂兄弟节点 II](https://leetcode.cn/problems/cousins-in-binary-tree-ii)
### 题目
给你一棵二叉树的根 root ，请你将每个节点的值替换成该节点的所有 **堂兄弟节点值的和** 。

堂兄弟: 为同一层节点, 但父节点不同, 则两个节点视为互为堂兄弟

### 分析
- 使用BFS遍历树(每次将子节点放入栈中, 不采用pop()方式取节点, 而采用设置tmp栈+清空栈的方式, 这样tmp栈中就保存了当前层的每一个节点了)
- 堂兄弟节点值的和 = 当前层所有节点和 - 同父节点的两个节点和
- 两次遍历当前层节点，去更新下一层子节点的值。(只有父节点才直到那些是自己的儿子)
    - 第一次遍历当前层节点: 获得下一层所有子节点的和
    - 第二次遍历当前层节点: 更新子节点的值


In [ ]:
def process(root):
    next_queue = [root]
    root.val = 0  # 根节点没有堂兄弟

    while next_queue:
        # 拿到当前层所有节点, 并清空下层节点栈
        current_queue = next_queue
        next_queue = []

        # 第一次遍历, 计算下一层所有节点和, 并将下一层节点放入下层栈
        children_sum = 0
        for node in current_queue:
            if node.left is not None:
                next_queue.append(node.left)
                children_sum += node.left.val
            if node.right is not None:
                next_queue.append(node.right)
                children_sum += node.right.val

        # 第二次遍历, 更新所有子节点的值
        for node in current_queue:
            # 获得自己孩子的和
            my_children = (node.left.val if node.left is not None else 0) + (node.right.val if node.right is not None else 0)
            # 更新子节点的值
            if node.left is not None:
                node.left.val = children_sum - my_children
            if node.right is not None:
                node.right.val = children_sum - my_children
